In [1]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
_query = """
SELECT
  trim('"' FROM block::text )::int as block,
  trim('"' FROM min::text )::int as min,
  s.holding,
  s.acks,
  s.msgqueue,
  s.inmsgqueue,
  s.apiqueue,
  s.ackqueue,
  s.timermsg,
  ts::text::time,
  s.run
FROM (SELECT
  --mt     | "LIST_SIZES Holding: %v, Acks: %v, MsgQueue: %v, InMsgQueue: %v, APIQueue: %v, AckQueue: %v, TimerMsg: %v "
  run,
  e->'log'->'height' as block,
  e->'log'->'min' as min,
  e->'log'->'event'->0 as holding,
  e->'log'->'event'->1 as acks,
  e->'log'->'event'->2 as msgqueue,
  e->'log'->'event'->3 as inmsgqueue,
  e->'log'->'event'->4 as apiqueue,
  e->'log'->'event'->5 as ackqueue,
  e->'log'->'event'->6 as timermsg,
  e->'log'->'event'->7 as fmt,
  e->'ts' as ts
  --e as l
FROM
  logs
) as s

WHERE
  s.fmt::varchar like '%LIST_SIZES%'
"""

_order = """
ORDER BY ts;
"""


def query(i):
    return _query + ' AND run = %s ' % i + _order

connection = pg.connect("host='localdb' dbname=load user=load password='load'")


In [10]:
log_runs = pd.read_sql_query('SELECT * from log_runs WHERE id in (3, 1);', con=connection)
log_runs

,id,label,ts
0,1,v5_test_LLLLLLLFF_d100,2019-01-31 17:07:33.116239
1,3,v6_test_LLLLLLLFF_d100,2019-01-31 17:09:05.469478


In [12]:
import plotly.plotly as py
import plotly.graph_objs as go

data = []
frames = []
for i, n in log_runs['label'].items():
    r = log_runs['id'][i]
    df = pd.read_sql_query(query(r), con=connection)
    frames.append(df)
    data.append(
        go.Scatter(x=df['ts'], y=df['inmsgqueue'], name=n.replace('test_','')+'inmsg')
    )

py.iplot(data, filename='WIP-queues')

In [13]:
s = (frames[0].sum()['inmsgqueue'] / 10, frames[1].sum()['inmsgqueue'] / 10)

100 * (s[0] - s[1]) / s[1] # Average % improvement

177.04249951484573

## Internal Message Queue


### Test Scenario: 7 Leaders 2 Followers

* Ran simulator pushing 10 sets of 1000 commit/reveal messages.
* Simulator is configured to drop 10% of network messages
* Network is configured in 'alot' configuration
* Each set is pushed into the API Queue first (without EC funding)
* Then the associated EC address is funded allowing messages to be processed.


### Result: Stability Gains

This graph shows an improvement to messages funneled through the Internal Message Queue.

This change allows for other types of network messages related to consensus 
to be handeled on time without being backloged by commit/reveal messages.

### Future Work: Minute Stretching

During testing we realized that under load, Internal End-Of-Minute EOM messages are being delayed causing the network to delay consensus. We've been calling this issue 'Minute Stretching'.

We should be able to take a similar approach to further improve EOM stability.

### Relate to Code

V6.1.1 adds an additional InternalMessageQueue2 for handling Commit & Reveal Messages
[code](https://github.com/FactomProject/factomd/blob/FD-771_rebased/engine/NetworkProcessorNet.go#L183-L186)
